# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
!pip install hvplot

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.68,64,10,1.79,NZ,1701746954
1,1,college,64.8569,-147.8028,-17.53,80,75,2.57,US,1701746955
2,2,bilibino,68.0546,166.4372,-25.87,100,100,1.16,RU,1701746956
3,3,bethel,41.3712,-73.4140,5.84,76,92,0.89,US,1701746958
4,4,constantia,44.1833,28.6500,1.50,68,73,4.52,RO,1701746959


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
plot1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles="OSM", size="Humidity", frame_height=900, frame_width=900, color="City", alpha=0.5,\
                                  xlabel="x", ylabel="y")

# Display the map
plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[(city_data_df['Max Temp'] >=20.00) &  (city_data_df['Max Temp'] <=27.00) & (city_data_df['Wind Speed'] < 4.5) & \
              (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,pisco,-13.7000,-76.2167,20.03,83,0,3.60,PE,1701747043
108,108,mermoz boabab,14.7065,-17.4758,24.88,83,0,4.12,SN,1701747059
115,115,guasdualito,7.2424,-70.7324,25.98,89,0,0.00,VE,1701747067
291,291,kedougou,12.5579,-12.1743,23.01,27,0,1.15,SN,1701747261
351,351,cabo san lucas,22.8909,-109.9124,24.17,69,0,2.06,MX,1701747064
376,376,torit,4.4133,32.5678,23.63,60,0,0.47,SS,1701747349
531,531,dakar,14.6937,-17.4441,25.07,83,0,4.12,SN,1701747503


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[['City', 'Country', "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
89,pisco,PE,-13.7000,-76.2167,83,
108,mermoz boabab,SN,14.7065,-17.4758,83,
115,guasdualito,VE,7.2424,-70.7324,89,
291,kedougou,SN,12.5579,-12.1743,27,
351,cabo san lucas,MX,22.8909,-109.9124,69,
376,torit,SS,4.4133,32.5678,60,
531,dakar,SN,14.6937,-17.4441,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {"categories": categories,
          "apiKey": geoapify_key,
          "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # print(latitude)
    # print(longitude)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # pprint(name_address)
    # break
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
mermoz boabab - nearest hotel: La maison blanche
guasdualito - nearest hotel: No hotel found
kedougou - nearest hotel: Auberge Thomas Sankara
cabo san lucas - nearest hotel: Comfort Rooms
torit - nearest hotel: Safari Link
dakar - nearest hotel: 翁记小吃


,City,Country,Lat,Lng,Humidity,Hotel Name
89,pisco,PE,-13.7000,-76.2167,83,La Portada
108,mermoz boabab,SN,14.7065,-17.4758,83,La maison blanche
115,guasdualito,VE,7.2424,-70.7324,89,No hotel found
291,kedougou,SN,12.5579,-12.1743,27,Auberge Thomas Sankara
351,cabo san lucas,MX,22.8909,-109.9124,69,Comfort Rooms
376,torit,SS,4.4133,32.5678,60,Safari Link
531,dakar,SN,14.6937,-17.4441,83,翁记小吃


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
plot2 = hotel_df.hvplot.points("Lng", 
                               "Lat", 
                               geo=True, 
                               tiles="OSM", 
                               frame_height=500, 
                               frame_width=700,
                               color="City", 
                               alpha=0.5, 
                               xlabel="x",
                               ylabel="y",
                               scale = 1,
                              hover_cols=["Hotel Name","Country"])

# Display the map
plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

In [ ]:
# Somehow, I do not see the hover message when I hover over any point. I connected with an AskBCS assistant, who confirmed my correct code. 
# But weren't sure why I do not see the hover message.